---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [1]:
!head Employee_Movie_Choices.txt

#Employee	Movie
Andy	Anaconda
Andy	Mean Girls
Andy	The Matrix
Claude	Anaconda
Claude	Monty Python and the Holy Grail
Claude	Snakes on a Plane
Frida	The Matrix
Frida	The Shawshank Redemption
Frida	The Social Network


In [11]:
def answer_one():    
    """
    Using NetworkX, load in the bipartite graph from Employee_Movie_Choices.txt and return that graph.
    """
    file_name = 'Employee_Movie_Choices.txt'
    
    # file contains two columns (emp name) and (movie title)
    G = nx.read_edgelist(path = file_name,
                         delimiter = '\t')
    
    return G

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [30]:
def answer_two():
    """
    Using the graph from the previous question, add nodes attributes named 'type' where
    movies have the value 'movie' and employees have the value 'employee' and return that graph.
    """
    # create graph
    G = answer_one()
    
    # assign attributes
    for node in G.nodes():
        if node in employees:
            G.add_node(node, type="employee")
        else:
            G.add_node(node, type="movie")
        
    
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [34]:
def answer_three():
    """
    Find a weighted projection of the graph from answer_two which tells us
    how many movies different pairs of employees have in common.
    """
    # create graph
    G = answer_two()
    
    # make projection
    G_p = nx.bipartite.weighted_projected_graph(G,employees)
    
    return G_p

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [126]:
def answer_four():
        
    """
    Suppose you'd like to find out if people that have a high
    relationship score also like the same types of movies.

    Find the Pearson correlation ( using DataFrame.corr() )
    between employee relationship scores and the number of movies
    they have in common. If two employees have no movies in common
    it should be treated as a 0, not a missing value, and should be included
    in the correlation calculation.
    """
    # load projected graph based on movie tastes
    G_p_movies = answer_three()
    
    # load graph based on relationships
    file_name = 'Employee_Relationships.txt'
    
    # file contains two columns (emp name) and (movie title)
    G_friend = nx.read_weighted_edgelist(path = file_name,
                         delimiter = '\t')
    
    # convert data into pandas DataFrame
    df1 = pd.DataFrame(G_p_movies.edges(data=True),columns=['From', 'To', 'movie_score'])
    df1['movie_score'] = df1['movie_score'].apply(lambda x: x['weight'])
    
    # restructure edges for uniqueness
    df1_copy = df1.copy()
    df1_copy['From_'] = df1_copy[['From','To']].min(axis=1)
    df1_copy['To_'] = df1_copy[['From','To']].max(axis=1)
    df1_copy = df1_copy.drop(['From','To'],axis=1)
    
    # convert data into pandas DataFrame
    df2 = pd.DataFrame(G_friend.edges(data=True),columns=['From', 'To', 'friend_score'])
    df2['friend_score'] = df2['friend_score'].apply(lambda x: x['weight'])
    
    # restructure edges for uniqueness
    df2_copy = df2.copy()
    df2_copy['From_'] = df2_copy[['From','To']].min(axis=1)
    df2_copy['To_'] = df2_copy[['From','To']].max(axis=1)
    df2_copy = df2_copy.drop(['From','To'],axis=1)
    
    # merge df
    df_merge = pd.merge(left=df1_copy,right=df2_copy,how='outer',left_on=['From_','To_'],right_on=['From_','To_'])
    df_merge = df_merge.fillna(0)
    
    return df_merge['movie_score'].corr(df_merge['friend_score'])